### Exercises

#### Exercise-1

Retrieve all `REVIEWED` relationships from the graph where the summary of the review contains the string fun, returning the movie title reviewed and the rating and summary of the relationship.

```
MATCH (:Person)-[r:REVIEWED]->(m:Movie)
WHERE toLower(r.summary) CONTAINS 'fun'
RETURN  m.title as Movie, r.summary as Review, r.rating as Rating
```

#### Exercise-2

Retrieve all people who have produced a movie, but have not directed a movie, returning their names and the movie titles.

```
MATCH (a:Person)-[:PRODUCED]->(m:Movie)
WHERE NOT ((a)-[:DIRECTED]->(:Movie))
RETURN a.name, m.title
```

#### Exercise-3

Retrieve the movies and their actors where one of the actors also directed the movie, returning the actors names, the director’s name, and the movie title.

```
MATCH (a1:Person)-[:ACTED_IN]->(m:Movie)<-[:ACTED_IN]-(a2:Person)
WHERE exists((a2)-[:DIRECTED]->(m))
RETURN  a1.name as Actor, a2.name as `Actor/Director`, m.title as Movie
```

#### Exercise-4

Write a Cypher query that retrieves all movies that *Gene Hackman* has acted it, along with the directors of the movies. In addition, retrieve the actors that acted in the same movies as *Gene Hackman*. Return the name of the movie, the name of the director, and the names of actors that worked with *Gene Hackman*.

```
MATCH (a1:Person)-[:ACTED_IN]->(m:Movie)<-[:ACTED_IN]-(a2:Person)
WHERE exists((a2)-[:DIRECTED]->(m))
RETURN  a1.name as Actor, a2.name as `Actor/Director`, m.title as Movie
```

#### Exercise-5

Retrieve all movies that *Tom Cruise* has acted in and the co-actors that acted in the same movie, returning the movie title and the list of co-actors that *Tom Cruise* worked with.

```
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)<-[:ACTED_IN]-(p2:Person)
WHERE p.name ='Tom Cruise'
RETURN m.title as movie, collect(p2.name) AS `co-actors`
```

#### Exercise-6

Retrieve the actors who have acted in exactly five movies, returning the name of the actor, and the list of movies for that actor.

```
MATCH (a:Person)-[:ACTED_IN]->(m:Movie)
WITH  a, count(m) AS numMovies, collect(m.title) AS movies
WHERE numMovies = 5
RETURN a.name, movies
```

#### Exercise-7

Write a Cypher query that retrieves all actors that acted in movies, and also retrieves the producers for those movies. During the query, collect the names of the actors and the names of the producers. Return the movie titles, along with the list of actors for each movie, and the list of producers for each movie making sure there is no duplication of data. Order the results returned based upon the size of the list of actors.

```
MATCH (a:Person)-[:ACTED_IN]->(m:Movie),
      (m)<-[:PRODUCED]-(p:Person)
WITH  m, collect(DISTINCT a.name) AS cast, collect(DISTINCT p.name) AS producers
RETURN DISTINCT m.title, cast, producers
ORDER BY size(cast)
```

#### Exercise-8

Write a Cypher query that retrieves all actors that acted in movies, and collects the list of movies for any actor that acted in more than five movies. Return the name of the actor and the list.

```
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WITH p, collect(m) AS movies
WHERE size(movies)  > 5
RETURN p.name, movies
```

Modify the query you just wrote so that before the query processing ends, you unwind the list of movies and then return the name of the actor and the title of the associated movie.

```
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WITH p, collect(m) AS movies
WHERE size(movies)  > 5
WITH p, movies UNWIND movies AS movie
RETURN p.name, movie.title
```

#### Exercise-9

Add the label *OlderMovie* to any Movie node that was released before 2010.

```
MATCH (m:Movie)
WHERE m.released < 2010
SET m:OlderMovie
RETURN DISTINCT labels(m)
```

#### Exercise-10

Create the *ACTED_IN* relationship between the actors, *Robin Wright*, *Tom Hanks*, and *Gary Sinise* and the movie, *Forrest Gump*.

```
MATCH (m:Movie)
WHERE m.title = 'Forrest Gump'
MATCH (p:Person)
WHERE p.name = 'Tom Hanks' OR p.name = 'Robin Wright' OR p.name = 'Gary Sinise'
CREATE (p)-[:ACTED_IN]->(m)
```
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
```
MATCH (p:Person), (m:Movie)
WHERE p.name IN ['Tom Hanks','Gary Sinise', 'Robin Wright']
      AND m.title = 'Forrest Gump'
MERGE (p)-[:ACTED_IN]->(m)
```

#### Exercise-11

Add the roles property to the three *ACTED_IN* relationships that you just created to the movie, *Forrest Gump* using this information: *Tom Hanks* played the role, *Forrest Gump*. *Robin Wright* played the role, *Jenny Curran*. *Gary Sinise* played the role, *Lieutenant Dan Taylor*.

```
MATCH (p:Person)-[rel:ACTED_IN]->(m:Movie)
WHERE m.title = 'Forrest Gump'
SET rel.roles =
CASE p.name
  WHEN 'Tom Hanks' THEN ['Forrest Gump']
  WHEN 'Robin Wright' THEN ['Jenny Curran']
  WHEN 'Gary Sinise' THEN ['Lieutenant Dan Taylor']
END
```

#### Exercise-12

Use `MERGE` to create (`ON CREATE`) a node of type *Movie* with the title property, *Forrest Gump*. If created, set the released property to 1994.

```
MERGE (m:Movie {title: 'Forrest Gump'})
ON CREATE SET m.released = 1994
RETURN m
```

#### Exercise-13

Use `MERGE` to update (`ON MATCH`) a node of type Movie with the title property, *Forrest Gump*. If found, set the tagline property to "Life is like a box of chocolates… you never know what you’re gonna get.".

```
MERGE (m:Movie {title: 'Forrest Gump'})
ON CREATE SET m.released = 1994
ON MATCH SET m.tagline = "Life is like a box of chocolates...you never know what you're gonna get."
RETURN m
```

#### Exercise-14

Add a parameter named year to your session with a value of 2000.

```
:param year => 2000
```

Write and execute a Cypher query that returns the names of people who reviewed movies and the actors in these movies by returning the name of the reviewer, the movie title reviewed, the release date of the movie, the rating given to the movie by the reviewer, and the list of actors for that particular movie. Also use the *year* parameter in this query.

```
MATCH (r:Person)-[rel:REVIEWED]->(m:Movie)<-[:ACTED_IN]-(a:Person)
WHERE m.released = $year
RETURN  DISTINCT r.name, m.title, m.released, rel.rating, collect(a.name)
```

Add a parameter named *ratingValue* to your session with a value of 65. Also, change the *year* parameter from 2000 to 2006.

```
:params {year: 2006, ratingValue: 65}
```

Modify the query you wrote previously to also filter the result returned by the *rating* for the movie.

```
MATCH (r:Person)-[rel:REVIEWED]->(m:Movie)<-[:ACTED_IN]-(a:Person)
WHERE m.released = $year AND
      rel.rating > $ratingValue
RETURN  DISTINCT r.name, m.title, m.released, rel.rating, collect(a.name)
```